In [1]:
from _ROTOR import rotor 
from _MISSION_PLANNER import MissionPlanner
import numpy as np
import matplotlib.pyplot as plt 

In [6]:
x = rotor()
x.set_rotor(5,0.5,3,0.8,0,twist0=np.deg2rad(5),twist_slope=0,omega=30)
lambdas,v,sigma,twist, thrust ,dT,Fs,d_torque, torque = x.bemt(V=5)


In [ ]:
import numpy as np
# Create rotor instance
x = rotor()
radius=  4
root_cut_out = 0.8
# Set more realistic rotor parameters
x.set_rotor(
    radius=radius,        # m
    root_cut_out=root_cut_out,  # m
    chord0=0.25,       # m, smaller chord for realistic solidity
    chord_slope= (0.1-0.25)/(radius-root_cut_out),
    blades=3,
    omega=25,           # rad/s (~238 RPM
    twist0= np.deg2rad(3),
    twist_slope= np.deg2rad(1-3)/(radius-root_cut_out)
)
x.atmosphere(h=2000)

(275.15, 94218.07928359076, 1.154950429497074, 332.5274883374305)

In [26]:
# Forward flight conditions
forward_speed = 55.55  # m/s
theta0 = np.deg2rad(8)
theta1c = np.deg2rad(5)
theta1s = np.deg2rad(0)
alpha_tpp = np.deg2rad(3)

# Compute forces and flapping angles
forces_moments, R_grid, PSI_grid, alpha, dT, dH, dY = x.forward(
    forward_speed=forward_speed,
    theta0=theta0,
    theta1c=theta1c,
    theta1s=theta1s,
    alpha_tpp=alpha_tpp
)

# Print relevant outputs
print("FZ (Thrust) [N]:", forces_moments['FZ'])
print("FX (Forward) [N]:", forces_moments['FX'])
print("FY (Lateral) [N]:", forces_moments['FY'])
print("beta0 [rad]:", forces_moments['beta0'])
print("beta1c [rad]:", forces_moments['beta1c'])
print("beta1s [rad]:", forces_moments['beta1s'])
print("CT (Rotor Thrust Coefficient):", forces_moments.get('CT', 'Not computed'))

FZ (Thrust) [N]: 43259.80642271439
FX (Forward) [N]: 94332.3569711001
FY (Lateral) [N]: 5195.928259166494
beta0 [rad]: 0.017453292519943295
beta1c [rad]: -0.034392305025982464
beta1s [rad]: -0.08400120948322674
CT (Rotor Thrust Coefficient): 0.02483877176865653
